**Import pyspark.sql**

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tugas_9').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/23 00:36:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/23 00:36:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/23 00:36:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### 1. Basic SQL Operations in Spark SQL

In [3]:
data = [
    ('James', 34, 'M', 3000, 1),
    ('Anna', 28, 'F', 4100, 2),
    ('Lee', 23, 'M', 2700, 1)
]
columns = ['Name', 'Age', 'Gender', 'Salary', 'DeptId']
df = spark.createDataFrame(data, schema=columns)
df.createOrReplaceTempView('employees')
spark.sql('SELECT * FROM employees').show()
spark.sql('SELECT Name, Age FROM employees WHERE Salary > 3000').show()
spark.sql('SELECT AVG(Salary) FROM employees').show()


+-----+---+------+------+------+
| Name|Age|Gender|Salary|DeptId|
+-----+---+------+------+------+
|James| 34|     M|  3000|     1|
| Anna| 28|     F|  4100|     2|
|  Lee| 23|     M|  2700|     1|
+-----+---+------+------+------+

+----+---+
|Name|Age|
+----+---+
|Anna| 28|
+----+---+

+------------------+
|       avg(Salary)|
+------------------+
|3266.6666666666665|
+------------------+



### 2. Advanced Queries for Data Analysis

In [4]:
spark.sql('''
SELECT Gender, SUM(Salary) as TotalSalary, Age
FROM employees
GROUP BY Gender, Age
ORDER BY Age
''').show()

+------+-----------+---+
|Gender|TotalSalary|Age|
+------+-----------+---+
|     M|       2700| 23|
|     F|       4100| 28|
|     M|       3000| 34|
+------+-----------+---+



In [7]:
# 1. Cari rata-rata gaji per departemen.
spark.sql('SELECT * FROM employees').show()
print('lihat rata rata per departemen')
spark.sql('''
SELECT deptId, AVG(Salary) as avgSalary
FROM employees
GROUP BY deptId
''').show()

+-----+---+------+------+------+
| Name|Age|Gender|Salary|DeptId|
+-----+---+------+------+------+
|James| 34|     M|  3000|     1|
| Anna| 28|     F|  4100|     2|
|  Lee| 23|     M|  2700|     1|
+-----+---+------+------+------+

lihat rata rata per departemen


+------+---------+
|deptId|avgSalary|
+------+---------+
|     1|   2850.0|
|     2|   4100.0|
+------+---------+



In [16]:
# 2. Temukan karyawan yang memiliki gaji di atas rata-rata untuk gender masing-masing.
spark.sql('SELECT * FROM employees').show()
print('Karyawan yg gajinya diatas rata rata untuk tiap gender')
spark.sql('''
SELECT Name, Age, Gender, Salary 
FROM employees
JOIN (
    SELECT Gender, AVG(Salary) AS avgSalary
    FROM employees
    GROUP BY Gender
) AS avgSalariesz
ON employees.Gender = avgSalaries.Gender
WHERE e.Salary > avgSalaries.avgSalary
''').show()


ConnectionRefusedError: [Errno 111] Connection refused